In [23]:
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index import set_global_service_context
from llama_index.storage.storage_context import StorageContext
from llama_index.schema import Document

import chromadb


In [20]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = "https://betatest.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "9e3202cf6a184049bd886bf2b8b0d456"

In [27]:
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.environ["OPENAI_API_KEY"],
    azure_endpoint=os.environ["OPENAI_API_BASE"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="gpt-35-turbo-0301",
    api_key=os.environ["OPENAI_API_KEY"],
    azure_endpoint=os.environ["OPENAI_API_BASE"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

set_global_service_context(service_context)


In [30]:
def create_embedding_store(name):
    chroma_client = chromadb.Client()
    return chroma_client.create_collection(name)
collection = create_embedding_store("web2")



In [45]:
file_content = open("./aws_ec2_instances.txt","r").read()
docs = [Document(text=file_content)]
vector_store = ChromaVectorStore(chroma_collection=collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)

In [59]:
instruction = """
You are expert in creating SQL Query for Postgres. Based on user query, create a SQL Query to fetch data from Postgres DB.
Your job is to convert TEXT to SQL Query.

Your job is to only give SQL Query. You don't need to execute it. We will execute it on our end. So please make sure that your SQL Query is correct. 
Your job is to only give one sql query. dont write any other text or information.

Example:
Instance count in each availability zone: select placement_availability_zone as az, instance_type, count(*) from aws_ec2_instance group by placement_availability_zone, instance_type;
List instances whose detailed monitoring is not enabled: select instance_id, monitoring_state from aws_ec2_instance where monitoring_state = 'disabled';
Count the number of instances by instance type: select instance_type, count(instance_type) as count from aws_ec2_instance group by instance_type;
List instances stopped for more than 30 days: select instance_id, instance_state, launch_time, state_transition_time from aws_ec2_instance where instance_state = 'stopped' and state_transition_time <= (current_date - interval '30' day);
List of instances without application tag key: select instance_id, tags from aws_ec2_instance where not tags :: JSONB ? 'application';
Get maintenance options for each instance: select instance_id, instance_state, launch_time, maintenance_options ->> 'AutoRecovery' as auto_recovery from aws_ec2_instance;
Get license details for each instance: select instance_id, instance_type, instance_state, l ->> 'LicenseConfigurationArn' as license_configuration_arn from aws_ec2_instance, jsonb_array_elements(licenses) as l;
List EC2 instances having termination protection safety feature enabled: select instance_id, disable_api_termination from aws_ec2_instance where not disable_api_termination;

User Query: List instances which are stopped for more than 30 days
"""

In [61]:
query_engine = index.as_query_engine()
answer = query_engine.query("Is Xbox better than PlayStation?")
print(answer)

I'm sorry, I cannot answer that question as it is not related to the context information provided about AWS EC2 instances and their corresponding SQL table.
